In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model and tokenizer...")
model_name = "databricks/dolly-v2-3b"
model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

/home/ubuntu/jsonformer/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model and tokenizer...
Loaded model and tokenizer


In [4]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

weather_schema = {
    "type": "object",
    "properties": {
        "temperature": {"type": "string"},
        "humidity": {
            "type": "string",
        },
        "wind_speed": {
            "type": "object",
            "properties": {
                "value": {"type": "string"},
                "unit": {"type": "string"},
            },
        },
    },
}

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=weather_schema,
    prompt="generate the weather",
    device="cuda",
)

print("Generating...")
output = builder()

highlight_values(output)


Generating...
{
  temperature: "22.0",
  humidity: "60",
  wind_speed: {
    value: "5",
    unit: "mph"
  }
}


In [3]:
car = {
    "type": "object",
    "properties": {
        "make": {"type": "string"},
        "model": {"type": "string"},
        "year": {"type": "string"},
        "colors": {
            "type": "array",
            "items": {"type": "string"},
        },
        "shouldUseTurnSignal": {"type": "boolean"},
    },
}

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car,
    prompt="generate an example car",
    device="cuda",
)

print("Generating...")
output = builder()

highlight_values(output)


Generating...
{
  make: "audi",
  model: "model a4",
  year: 1.0,
  colors: [
    "blue"
  ]
}
